In [46]:
import numpy as np 
import tensorflow as tf
    
tf.reset_default_graph()
sess = tf.InteractiveSession()

def unpickle( file ):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict
 
data = unpickle( 'cifar-10-batches-py/data_batch_1' )

features = data['data']
labels = data['labels']

one_hot_labels = []
for label in labels:
    one_hot = np.zeros(10)
    one_hot.put(label,1)
    one_hot_labels.append(one_hot)


def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.2)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv( x, filter_size=3, stride=1, num_filters=64, is_output=False, name="conv" ):
    
    with tf.name_scope(name) as scope:
        #W = weight_variable([filter_size, filter_size, x.get_shape().as_list()[3], num_filters])
        W = tf.Variable( 1e-3*np.random.randn(filter_size, filter_size, x.get_shape().as_list()[3], num_filters).astype(np.float32))
        b = bias_variable([num_filters])
        h = []
        
        if not is_output:
            h = tf.nn.relu(tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='SAME') + b)
        else:
            h = tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='SAME') + b
        return h


def fc( x, out_size=50, is_output=False, name="fc" ):
    with tf.name_scope(name) as scope:
        #W = weight_variable( [x.get_shape().as_list()[1], out_size])
        W = tf.Variable( 1e-3*np.random.randn(x.get_shape().as_list()[1], out_size).astype(np.float32))
        b = bias_variable([out_size])
        h = []
        if not is_output:
            h = tf.nn.relu(tf.matmul(x,W)+b)
        else:
            h = tf.matmul(x,W)+b
        return h

x = tf.placeholder(tf.float32, shape=[None, 32*32*3], name="images")
label_ = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1,32,32,3])

h0 = conv(x_image)
h1 = conv(h0,stride=2)
h2 = conv(h1,stride=2)
shape = h2.get_shape().as_list()

flat = tf.reshape(h2,[-1,shape[1]*shape[2]*shape[3]])

fc1 = fc(flat)
fc2 = fc(fc1,out_size=10,is_output=True)

label_conv = tf.nn.softmax(fc2)


with tf.name_scope('Cost') as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(label_ * tf.log(tf.clip_by_value(label_conv,1e-10,1)), reduction_indices=[1]))
    
with tf.name_scope('Accuracy') as scope:
    correct_prediction = tf.equal(tf.argmax(label_conv,1), tf.argmax(label_,1))
    label_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  
with tf.name_scope('Optimizer') as scope:
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
acc_summary = tf.summary.scalar( 'Accuracy', label_acc )
cost_summary = tf.summary.scalar( 'Cost', cross_entropy )

merged_summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter("./tf_logs",graph=sess.graph)

sess.run(tf.global_variables_initializer())

NUM_EPOCHS = 1000

for i in range(NUM_EPOCHS):
    train_step.run(feed_dict={x:features[:50], label_:one_hot_labels[:50]})
    summary_str,l = sess.run([merged_summary_op, label_acc],feed_dict={x:features[50:70], label_:one_hot_labels[50:70]})
    
    print("%d, %g"%(i,l))
    summary_writer.add_summary(summary_str,i)

0, 0.05
1, 0.05
2, 0.05
3, 0.05
4, 0.05
5, 0.05
6, 0.05
7, 0.05
8, 0.05
9, 0.05
10, 0.05
11, 0.05
12, 0.05
13, 0.05
14, 0.05
15, 0.05
16, 0.05
17, 0.05
18, 0.2
19, 0.2
20, 0.2
21, 0.2
22, 0.2
23, 0.2
24, 0.2
25, 0.2
26, 0.2
27, 0.2
28, 0.2
29, 0.2
30, 0.2
31, 0.2
32, 0.2
33, 0.2
34, 0.2
35, 0.2
36, 0.2
37, 0.2
38, 0.2
39, 0.2
40, 0.2
41, 0.2
42, 0.2
43, 0.2
44, 0.2
45, 0.2
46, 0.2
47, 0.2
48, 0.2
49, 0.2
50, 0.2
51, 0.2
52, 0.2
53, 0.2
54, 0.2
55, 0.2
56, 0.2
57, 0.2
58, 0.2
59, 0.2
60, 0.2
61, 0.2
62, 0.2
63, 0.2
64, 0.2
65, 0.2
66, 0.2
67, 0.2
68, 0.2
69, 0.2
70, 0.2
71, 0.2
72, 0.2
73, 0.2
74, 0.2
75, 0.1
76, 0.1
77, 0.05
78, 0.1
79, 0.1
80, 0.1
81, 0.15
82, 0.1
83, 0.1
84, 0.1
85, 0.1
86, 0.1
87, 0.1
88, 0.15
89, 0.2
90, 0.15
91, 0.15
92, 0.25
93, 0.15
94, 0.1
95, 0.2
96, 0.15
97, 0.2
98, 0.15
99, 0.2
100, 0.15
101, 0.15
102, 0.2
103, 0.2
104, 0.2
105, 0.2
106, 0.15
107, 0.15
108, 0.15
109, 0.15
110, 0.2
111, 0.15
112, 0.15
113, 0.2
114, 0.2
115, 0.15
116, 0.15
117, 0.2
118, 0.2
11

917, 0.2
918, 0.2
919, 0.2
920, 0.2
921, 0.2
922, 0.2
923, 0.2
924, 0.2
925, 0.2
926, 0.2
927, 0.2
928, 0.2
929, 0.2
930, 0.2
931, 0.2
932, 0.2
933, 0.2
934, 0.2
935, 0.2
936, 0.2
937, 0.2
938, 0.2
939, 0.2
940, 0.2
941, 0.2
942, 0.2
943, 0.2
944, 0.2
945, 0.2
946, 0.2
947, 0.2
948, 0.2
949, 0.2
950, 0.2
951, 0.2
952, 0.2
953, 0.2
954, 0.2
955, 0.2
956, 0.2
957, 0.2
958, 0.2
959, 0.2
960, 0.2
961, 0.2
962, 0.2
963, 0.2
964, 0.2
965, 0.2
966, 0.2
967, 0.2
968, 0.2
969, 0.2
970, 0.2
971, 0.2
972, 0.2
973, 0.2
974, 0.2
975, 0.2
976, 0.2
977, 0.2
978, 0.2
979, 0.2
980, 0.2
981, 0.2
982, 0.2
983, 0.2
984, 0.2
985, 0.2
986, 0.2
987, 0.2
988, 0.2
989, 0.2
990, 0.2
991, 0.2
992, 0.2
993, 0.2
994, 0.2
995, 0.2
996, 0.2
997, 0.2
998, 0.2
999, 0.2
